In [1]:
import json
import pandas as pd # type: ignore
import numpy as np # type: ignore
import sys, os
import datetime as dt

import comm_helper as ch # type: ignore

sys.path.append(os.environ["HOME"] + "/anchorage/source/python/lib/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility # type: ignore
import utils as aql_utils # type: ignore

sys.path.append(os.environ["HOME"]+"/trading/python/")
from lib import aplo
from lib import talos_utils

sys.path.append(os.environ["HOME"]+"/trading/keys/")
import keys

pd.options.display.max_columns = 50
pd.options.display.float_format = '{:,.8f}'.format

In [2]:
# Aplo client init
aplo_key = keys.aplo_read_key
ch.aplo_client = aplo.AploClient(aplo_key, debug=True)

# Talos client init
talos_api = keys.talos_api_v2
talos_secret_api = keys.talos_secret_v2
host = keys.talos_host
ch.talos = talos_utils.Talos(talos_api, talos_secret_api, host)

talos_adv_api = keys.talos_api_key_wl
talos_adv_secret_api = keys.talos_api_secret_wl
host_wl = keys.talos_host_wl
ch.talos_wl = talos_utils.Talos(talos_adv_api, talos_adv_secret_api, host_wl)

# Google Sheet init
os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "Order Update Script Sheet"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)

[2024-11-02T10:25:50.873+0000] {aplo.py:29} INFO - Debugging initialized for AploClient


## Google Spreadsheet fetching examples
> Link to the spreadsheet: https://docs.google.com/spreadsheets/d/1anmDOestfE9277fsR0jE2vLATGYqmnaiBUJRjU3-y6I/edit?gid=1262777392#gid=1262777392

### Microstrategy

In [3]:
tab_name = "MSTR_new_logic"

df = gsu._get_current_sheet_df(tab_name, 0)
df["venue_tuple"] = df["venue"].apply(lambda x: tuple(map(str, x.split(','))))
df.set_index("order_id", inplace=True)
df_dict = df.to_dict(orient="index")

mstr_orders_comms_df = ch.calculate_comms(order_details=df_dict, commission = 7) 
mstr_update = ch.generate_order_update_text(mstr_orders_comms_df, show_finished_orders=True)
print(mstr_update)

Order Update

Open Orders:

Cumulatively:
MicroStrategy Incorporated bought 4,204.45776686 BTC @ $61,363.44192433 for $258,000,000.00 net

Filled Orders:

Order 1- $100,000,000 TWAP with 63,000 limit price
MicroStrategy Incorporated bought 1,664.39196874 BTC @ $60,082.00104186 for $100,000,000.00 net

Order 2- $5,000,000 Limit with 60,500 limit price
MicroStrategy Incorporated bought 82.61530804 BTC @ $60,521.47136588 for $5,000,000.00 net

Order 3- $5,000,000 Limit with 60,400 limit price
MicroStrategy Incorporated bought 82.72161911 BTC @ $60,443.69118936 for $5,000,000.00 net

Order 4- $5,000,000 Limit with 60,300 limit price
MicroStrategy Incorporated bought 82.87434684 BTC @ $60,332.30053150 for $5,000,000.00 net

Order 5- $5,000,000 Limit with 60,200 limit price
MicroStrategy Incorporated bought 82.99643880 BTC @ $60,243.54866315 for $5,000,000.00 net

Order 6- $40,000,000 TWAP with 63,000 limit price
MicroStrategy Incorporated bought 651.57458258 BTC @ $61,389.74887825 for $40,0

### Aplo

In [8]:
tab_name = "a16z_FLOW"

df = gsu._get_current_sheet_df(tab_name, 0)
df["venue_tuple"] = df["venue"].apply(lambda x: tuple(map(str, x.split(','))))
df.set_index("order_id", inplace=True)
df_dict = df.to_dict(orient="index")

# In this example, since the MSTR tab does not have a commission column, we use the commission provided in the function call of 7 bps

orders_comms_df = ch.calculate_comms(order_details=df_dict, commission=40) 
order_update = ch.generate_order_update_text(orders_comms_df, show_finished_orders=False)
print(order_update)

[2024-11-02T12:51:07.289+0000] {connectionpool.py:823} WARNING - Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))': /v1/orders?OrderID=91ab1e9d-d290-44c9-aebf-81660aa6e12f
Order Update

Open Orders:
CNK Fund III, L.P. sold 147,019.47989300 FLOW @ 0.50873840 for 74,794.45496960 net

Cumulatively:
CNK Fund III, L.P. sold 1,716,719.47989300 FLOW @ 0.52123614 for 894,816.23131862 net


In [11]:
orders_comms_df

,ClientName,OrderDesc,trade_side,bought_amount,bought_asset,sold_amount,sold_asset,commission,comms_ccy,price_with_comms,gross_price,amount_gross,OrderType,OrdStatus,Symbol,Date,Commission,print_output
4173132017611597,NIV Voyager Master Fund LP,,sell,"177,293.76796780",USDT,147.18880000,MKR,355.29813220,USDT,"1,204.53300773","1,206.94690153","177,649.06610000",TWAP,Open,MKR-USDT,2024-10-18 20:21:38,20.00000000,TRUE


In [4]:
tab_name = "a16z_SUI"

df = gsu._get_current_sheet_df(tab_name, 0)
df["venue_tuple"] = df["venue"].apply(lambda x: tuple(map(str, x.split(','))))
df.set_index("order_id", inplace=True)
df_dict = df.to_dict(orient="index")

# In this example, since the MSTR tab does not have a commission column, we use the commission provided in the function call of 7 bps

orders_comms_df = ch.calculate_comms(order_details=df_dict) 
orders_comms_df["bought_asset"] = orders_comms_df["bought_asset"].replace("USDT", "USD")
orders_comms_df["Symbol"] = orders_comms_df["Symbol"].str.replace('USDT', 'USD', regex=False)
order_update = ch.generate_order_update_text(orders_comms_df, show_finished_orders=False)
print(order_update)

/tmp/ipykernel_300492/2325697933.py:6 UserWarning: DataFrame columns are not unique, some columns will be omitted.

Order Update

Open Orders:

Cumulatively:
CNK Fund III, L.P. sold 32,805,545.91766012 SUI @ $1.88267171 for $61,762,073.19 net


In [9]:
orders_comms_df.iloc[:,2:]

,trade_side,bought_amount,bought_asset,sold_amount,sold_asset,commission,comms_ccy,price_with_comms,gross_price,amount_gross,OrderType,OrdStatus,Symbol,Date,Commission,print_output
07190ff5-427d-4011-94f1-a83f72e98170,sell,"100,000.00000000",USD,"52,078.05835303",SUI,155.76687444,SUI,1.92019448,1.92595506,"51,922.29147860",TWAP,Done,SUI-USD,2024-10-08 23:34:57,30.00000000,TRUE
f5bbd469-89ff-4746-a5ed-0086c2855c6f,sell,"5,368,750.00000000",USD,"2,765,301.48585753",SUI,"4,141.73961936",SUI,1.94147004,1.94438225,"2,761,159.74623817",TimeSliced,Done,SUI-USD,2024-10-08 23:37:29,15.00000000,TRUE
2518504434760136,sell,"2,433,284.92720668",USD,"1,282,260.00000000",SUI,"9,772.22862332",USDT,1.89765331,1.90527440,"2,443,057.15583000",TWAP,Done,SUI-USD,2024-10-09 09:14:58,40.00000000,TRUE
3f1228fd-7153-4d63-ae26-4513d55f08d6,sell,"2,457,498.99040514",USD,"1,282,060.00000000",SUI,"7,394.68101426",USDT,1.91683618,1.92260399,"2,464,893.67141940",TWAP,Done,SUI-USD,2024-10-09 12:06:39,30.00000000,TRUE
2854461089925803,sell,"2,863,431.74352240",USD,"1,535,000.00000000",SUI,"11,499.72587760",USDT,1.86542785,1.87291952,"2,874,931.46940000",TWAP,Done,SUI-USD,2024-10-09 19:20:01,40.00000000,TRUE
c150388e-6df1-4604-97c0-9ad0cd419d20,sell,"1,777,034.34000000",USD,"953,997.75442216",SUI,"2,853.43296437",SUI,1.86272382,1.86831199,"951,144.32145779",TWAP,Done,SUI-USD,2024-10-09 19:30:32,30.00000000,TRUE
1ed190bd-b5a1-456d-9d58-d343778f9f4b,sell,"1,873,171.79534000",USD,"1,000,000.00000000",SUI,"5,636.42466000",USDT,1.87317180,1.87880822,"1,878,808.22000000",TWAP,Done,SUI-USD,2024-10-09 23:36:26,30.00000000,TRUE
9a5702c9-1d8f-4d28-bbeb-46a2e31a63b2,sell,"744,578.46722800",USD,"400,000.00000000",SUI,"2,240.45677200",USDT,1.86144617,1.86704731,"746,818.92400000",TWAP,Done,SUI-USD,2024-10-10 03:23:13,30.00000000,TRUE
4497539430928936,sell,"2,790,157.66318740",USD,"1,500,000.00000000",SUI,"11,205.45246260",USDT,1.86010511,1.86757541,"2,801,363.11565000",TWAP,Done,SUI-USD,2024-10-10 04:40:15,40.00000000,TRUE
1692801265485113,sell,"2,725,539.12029784",USD,"1,500,000.00000000",SUI,"10,945.94024216",USDT,1.81702608,1.82432337,"2,736,485.06054000",TWAP,Done,SUI-USD,2024-10-10 08:48:31,40.00000000,TRUE


## Individual order fetching examples

### Talos, single order, no details

In [13]:
order_id = "eee15c9c-30bc-4d59-a9d8-e668fe91ca60"
client_name = "Harish"
venue = ("talos", "principal")

orders_comms_df = ch.calculate_comms(order_details=order_id, client_name=client_name, venue=venue, commission=10)
order_update = ch.generate_order_update_text(orders_comms_df, show_finished_orders=True)
print(order_update)

Order Update

Open Orders:

Cumulatively:
Harish sold 82,010.00000000 STRK @ $0.42957091 for $35,229.11 net

Filled Orders:
Harish sold 82,010.00000000 STRK @ $0.42957090 for $35,229.11 net



In [14]:
orders_comms_df

,ClientName,OrderDesc,trade_side,bought_amount,bought_asset,sold_amount,sold_asset,commission,comms_ccy,price_with_comms,gross_price,amount_gross,OrderType,OrdStatus,Symbol,Date,Commission,print_output
eee15c9c-30bc-4d59-a9d8-e668fe91ca60,Harish,,sell,"35,229.11000000",USD,"82,010.00000000",STRK,35.26000000,USD,0.42957090,0.43000090,"35,264.37380900",TWAP,Done,STRK-USD,2024-10-21 00:03:32,10,True


### Talos WL, single order, no details

In [4]:
order_id = "c960bfd9-2f08-4c92-8c57-579108134942"
client_name = "Three Arrows Capital Ltd. in liquidation"
venue = ("talos", "wl")

orders_comms_df = ch.calculate_comms(order_details=order_id, client_name=client_name, venue=venue, commission=40)
order_update = ch.generate_order_update_text(orders_comms_df, show_finished_orders=False)
print(order_update)

Order Update

Open Orders:
Three Arrows Capital Ltd. in liquidation sold 815,610.06550000 NEON @ 0.40887051 for 333,478.90344212 net

Cumulatively:
Three Arrows Capital Ltd. in liquidation sold 815,610.06550000 NEON @ 0.40887051 for 333,478.90324637 net


### Aplo, single order with description

In [6]:
order_id = "538434344162602"
client_name = "Teneo"
order_start_time = "2024-10-18 22:47:47" # This is the time that is displayed directly in Aplo, you can use your own local time
timezone = "Europe/Lisbon" # The ones we use the most are "Europe/Lisbon", "UTC" and "Us/Eastern"
aplo_ts = int(pd.to_datetime(order_start_time).tz_localize(timezone).tz_convert("UTC").timestamp() * 1000)
description = "Order 1 - XYZ units of JTO with $2.00 limit price"
venue = ("aplo", "default")

orders_comms_df = ch.calculate_comms(order_details=order_id, client_name=client_name, venue=venue, aplo_ts = aplo_ts, commission=0)
order_update = ch.generate_order_update_text(orders_comms_df, show_finished_orders=False)
print(order_update)

Order Update

Open Orders:

Cumulatively:
Teneo sold 1,093,166.67000000 STRK @ 0.40598391 for 443,808.08440200 net


In [7]:
orders_comms_df

,ClientName,OrderDesc,trade_side,bought_amount,bought_asset,sold_amount,sold_asset,commission,comms_ccy,price_with_comms,gross_price,amount_gross,OrderType,OrdStatus,Symbol,Date,Commission,print_output
538434344162602,Teneo,,sell,"443,808.08440200",USDT,"1,093,166.67000000",STRK,0.00000000,USDT,0.40598391,0.40598391,"443,808.08440200",TWAP,Done,STRK-USDT,2024-10-18 21:47:47,0,True


## Multiple orders fetching with list

### Talos Multiple orders
> This way you can't add descriptions or have multiple client names

In [7]:
order_id = ["fb4f4ceb-8e83-4cd8-b375-71fe10f30779", "8fb7208c-e814-4004-ae3c-1e3bc5509bdc"]
client_name = "Oblique Arrow"
venue = ("talos", "principal")

orders_comms_df = ch.calculate_comms(order_details=order_id, client_name=client_name, venue=venue, filled_and_cancelled_threshold=1)
order_update = ch.generate_order_update_text(orders_comms_df, show_finished_orders=True)
print(order_update)

Order Update

Open Orders:

Cumulatively:
Oblique Arrow bought 0.97580662 BTC @ $61,824.68817438 for $60,328.94 net
Oblique Arrow bought 12.19576762 ETH @ $2,473.52613955 for $30,166.55 net

Filled Orders:
Oblique Arrow bought 0.97580662 BTC @ $61,824.69277513 for $60,328.94 net
Oblique Arrow bought 12.19576762 ETH @ $2,473.52593754 for $30,166.55 net



In [8]:
orders_comms_df

,ClientName,OrderDesc,trade_side,bought_amount,bought_asset,sold_amount,sold_asset,commission,comms_ccy,price_with_comms,gross_price,amount_gross,OrderType,OrdStatus,Symbol,Date,Commission,print_output
fb4f4ceb-8e83-4cd8-b375-71fe10f30779,Oblique Arrow,,buy,0.97580662,BTC,"60,328.94000000",USD,0.00097678,BTC,"61,824.69277513","61,762.86808235",0.97678340,TWAP,Done,BTC-USD,2024-09-09 14:38:49,10,True
8fb7208c-e814-4004-ae3c-1e3bc5509bdc,Oblique Arrow,,buy,12.19576762,ETH,"30,166.55000000",USD,0.01220798,ETH,"2,473.52593754","2,471.05241160",12.20797560,TWAP,Done,ETH-USD,2024-09-09 14:39:19,10,True


## Order fetching with dictionary outside of Google Spreadsheets

### Talos, with description, default comms

In [13]:
order_dictionary = {
    "257921d9-b6a4-4d45-bf2d-2d095fdae1bd": {
        "client_name": "MicroStrategy Incorporated",
        "order_desc": "Order 1- $6,000,000 TWAP with 59,000 limit price",
        "print_output": "TRUE",
        "venue_tuple": ("talos", "principal"),
    },
    "80304403-f133-42ab-98cf-0302cb2ebf65": {
        "client_name": "MicroStrategy Incorporated",
        "order_desc": "Order 2- $5,000,000 with 58,300 limit price",
        "print_output": "TRUE",
        "venue_tuple": ("talos", "principal"),
    },
}

orders_comms_df = ch.calculate_comms(order_details=order_dictionary) 
order_update = ch.generate_order_update_text(orders_comms_df, show_finished_orders=True)
print(order_update)

Order Update

Open Orders:

Cumulatively:
MicroStrategy Incorporated bought 188.93606115 BTC @ $58,220.75432845 for $11,000,000.00 net

Filled Orders:

Order 1- $6,000,000 TWAP with 59,000 limit price
MicroStrategy Incorporated bought 103.23149984 BTC @ $58,121.79430774 for $6,000,000.00 net

Order 2- $5,000,000 with 58,300 limit price
MicroStrategy Incorporated bought 85.70456131 BTC @ $58,339.95208261 for $5,000,000.00 net



### Talos and Aplo, with mixed description and custom comms

In [14]:
order_dictionary = {
    "1231921241041743": {
        "order_start_time": "2024-09-17 23:28:41",
        "timezone": "Europe/Lisbon",
        "client_name": "Ribbit Capital",
        "order_desc": "",
        "commission": "15",
        "print_output": "TRUE",
        "venue_tuple": ("aplo", "default"),
    },
    "db20a9d0-9e13-4acf-98da-899715dbe643": {
        "order_start_time": "",
        "timezone": "",
        "client_name": "CNK Fund III, L.P.",
        "order_desc": "2024-08-29 7,398,709 SUI limit 0.85",
        "commission": "",
        "print_output": "TRUE",
        "venue_tuple": ("talos", "wl"),
    },
    "257921d9-b6a4-4d45-bf2d-2d095fdae1bd": {
        "client_name": "MicroStrategy Incorporated",
        "order_desc": "Order 1- $6,000,000 TWAP with 59,000 limit price",
        "commission": "7",
        "print_output": "TRUE",
        "venue_tuple": ("talos", "principal"),
    },
}

orders_comms_df = ch.calculate_comms(order_details=order_dictionary) 
order_update = ch.generate_order_update_text(orders_comms_df, show_finished_orders=True)
print(order_update)

Order Update

Open Orders:
Ribbit Capital bought 348,119.00000000 JTO @ 2.01999319 for 703,198.00930961 net

Cumulatively:
MicroStrategy Incorporated bought 103.26250029 BTC @ $58,104.34555768 for $6,000,000.00 net
Ribbit Capital bought 348,119.00000000 JTO @ 2.01999319 for 703,198.00988905 net
CNK Fund III, L.P. sold 179,962.02638400 SUI @ $0.84965008 for $152,904.75 net

Filled Orders:

Order 1- $6,000,000 TWAP with 59,000 limit price
MicroStrategy Incorporated bought 103.26250029 BTC @ $58,104.34555532 for $6,000,000.00 net

2024-08-29 7,398,709 SUI limit 0.85
CNK Fund III, L.P. sold 179,962.02638400 SUI @ $0.84965007 for $152,904.75 net



> Like before you can see that each order has its own commission

In [15]:
orders_comms_df

,ClientName,OrderDesc,trade_side,bought_amount,bought_asset,sold_amount,sold_asset,commission,comms_ccy,price_with_comms,gross_price,amount_gross,OrderType,OrdStatus,Symbol,Date,Commission,print_output
1231921241041743,Ribbit Capital,,buy,"348,119.00000000",JTO,"703,198.00988905",USDT,"1,053.21718905",USDT,2.01999319,2.01696774,"702,144.79270000",TWAP,Open,JTO-USDT,2024-09-17 22:28:18,15.00000000,TRUE
db20a9d0-9e13-4acf-98da-899715dbe643,"CNK Fund III, L.P.","2024-08-29 7,398,709 SUI limit 0.85",sell,"152,904.75000000",USD,"179,962.02638400",SUI,153.06000000,USD,0.84965007,0.85050057,"153,057.80601795",SteadyPace,Done,SUI-USD,2024-08-29 01:06:39,10.00000000,TRUE
257921d9-b6a4-4d45-bf2d-2d095fdae1bd,MicroStrategy Incorporated,"Order 1- $6,000,000 TWAP with 59,000 limit price",buy,103.26250029,BTC,"6,000,000.00000000",USD,0.07233438,BTC,"58,104.34555532","58,063.67251343",103.33483468,SteadyPace,Done,BTC-USD,2024-09-12 14:25:25,7.00000000,TRUE
